In [41]:
import pandas as pd
import numpy as np
import os
from joblib import Parallel, delayed

pd.options.display.max_rows=999
pd.options.display.max_columns=999

def align(s1,s2,debug=False,verbose=False):
    assert len(s1) > 0 and len(s2) > 0
    
    m = len(s1)
    n = len(s2)
    
    # for debugging
    if debug:
        global dp
    
    dp = np.zeros((m+1,n+1),dtype=np.int32)
    
    for i in range(1,m+1):
        for j in range(1,n+1):
            if s1[i-1:i] == s2[j-1:j]:
                dp[i][j] = max(dp[i-1][j-1] + 1, max(dp[i-1][j]-2, dp[i][j-1]-2))
            else:
                dp[i][j] = max(dp[i-1][j-1] - 1, max(dp[i-1][j]-2, dp[i][j-1]-2))
        
    # we want whole of s2 to be aligned
    # initialize j as last col
    # initialize i as max row
    max_row_val = np.NINF
    max_i = np.nan
    for irow in range(1,len(s1)+1):
        if dp[irow][n] > max_row_val:
            max_row_val = dp[irow][n]
            max_i = irow
    
    max_col_val = np.NINF
    max_j = np.nan
    for icol in range(1,len(s2)+1):
        if dp[m][icol] > max_col_val:
            max_col_val = dp[m][icol]
            max_j = icol
    
    if verbose:
        print(f'max_row_val: {max_row_val}, max_i: {max_i}')
        print(f'max_col_val: {max_col_val}, max_j: {max_j}')
    
    # keep track of bit mask
    mask1 = np.zeros((m,),dtype=np.byte)
    mask2 = np.zeros((n,),dtype=np.byte)

    # decide whether to start from last row or last col
    # initialize i and j accordingly
    # at the same time add starting cell to mask
    if verbose:
        print('Attempting semi-global alignment. \
        \Aim to align all of s1 against s2, \
        \or all of s2 against s1, whichever is longer.')
    if max_row_val >= max_col_val:
        i = max_i
        j = len(s2)
    else:
        i = len(s1)
        j = max_j
    
    # score for best path
    score = 0
    
    if debug:
        if dp[i][j] < 0:
            # no semi-global alignment found
            if verbose:
                print('No semi-global alignment found. \\
                Attempting to find a local alignment...')
            # try to find a cell with higher value than dp[i][j]
            # if its positive, update i and j
            # otherwise return empty results
            max_local_val = dp[i][j]
            max_local_i = np.nan
            max_local_j = np.nan
            for irow in range(2,m):
                for icol in range(2,n):
                    if dp[irow][icol] > max_local_val:
                        max_local_val = dp[irow][icol]
                        max_local_i = irow
                        max_local_j = icol
            if dp[max_local_i][max_local_j] > dp[i][j]:
                if verbose:
                    print('Local alignment found. Going ahead with that.') 
                # update i and j
                i = max_local_i
                j = max_local_j
            else:
                if verbose:
                    print('No local alignment found. Returning empty masks and negative INF') 
                return mask1, mask2, np.NINF
            
    mask1[i-1] = 1
    mask2[j-1] = 1
    
    if verbose:
        print(f'Assigned 1 at index {i-1} of mask1 and index {j-1} of mask2.')
    
    while i != 0:
        while j != 0:
            curr = dp[i][j]
            diag = dp[i-1][j-1] 
            left = dp[i][j-1] 
            abov = dp[i-1][j]
            
            score += curr
            if diag >= left and diag >= abov:
                mask1[i-1] = 1
                mask2[j-1] = 1
                if max(diag, max(left, abov)) < 0:
                    if verbose:
                        print(f'exiting bit masking at {i},{j} as additional step will reduce score')
                    break
                i = i - 1
                j = j - 1
            elif left >= diag and left >= abov:
                j = j - 1
            elif abov >= left and abov >= diag:
                i = i - 1
            else:
                print(f'unhandled error for {i},{j}')
                break
        break
     
    return mask1,mask2,score


def align_all(input_df):
    
    df = pd.DataFrame()

    def for_joblib(series):
        seqID = series.seqID
        AF_sequence = series.AF_sequence
        jpred_sequence = series.jpred_sequence
        AF_mask,jpred_mask,score = align(AF_sequence,jpred_sequence,debug=True,verbose=False)
        return df.append([{'seqID' : seqID, 
                           'score' : score, 
                           'AF_sequence' : AF_sequence,
                           'AF_mask' : AF_mask, 
                           'jpred_sequence' : jpred_sequence,
                           'jpred_mask' : jpred_mask}])

    df_list = Parallel(n_jobs=-1,verbose=1)(
        delayed(for_joblib)(row) for _,row in input_df.iterrows()
    )
    df = pd.concat(df_list, ignore_index=True)
    
    return df

def split(word):
    return [c for c in word]

def compress(seq,mask):
    return ''.join([s for s, m in zip(seq,mask) if m])

def string(byte_array):
    return ''.join(byte_array.astype(str))

def pad_jpred(debug=False):
    global AF_mask, jpred_mask, AF_sequence, jpred_sequence
    i = 0
    j = 0
    lower = []
    while i < len(AF_mask): # and j < len(jpred_align):
        j = min(j, len(jpred_mask)-1)
        if AF_mask[i] and jpred_mask[j]:
            lower.append(jpred_sequence[j])
            i += 1
            j += 1
        elif AF_mask[i] and not jpred_mask[j]:
            if debug:
                # for case 24716
                j += 1
            else:
                lower.append('-')
                j += 1
        elif not AF_mask[i]:
            lower.append('-')
            i += 1
        else:
            print('unhandled error')
            break
    return lower

In [42]:
os.chdir('/cluster/gjb_lab/2472402/.')
df = pd.read_csv('summary_table.csv')

aligned_df = align_all(df[:72])

it = aligned_df.iterrows()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 64 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  72 | elapsed:    1.8s remaining:    5.3s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:    5.8s finished


In [56]:
i, (seqID, score, AF_sequence, AF_mask, jpred_sequence, jpred_mask) = it.__next__()
print(seqID)
print(score)
#print(AF_sequence)
#print(string(AF_align))
#print(jpred_sequence)
#print(string(jpred_align))
#print(compress(AF_sequence,AF_align))
#print(compress(jpred_sequence,jpred_align))
#print(' '*100)
print(AF_sequence)
print(''.join(pad_jpred(debug=True)))
print(jpred_sequence)

24730
13191
MRILVLGVGNILLTDEAIGVRIVEALEQRYILPDYVEILDGGTAGMELLGDMANRDHLIIADAIVSKKNAPGTMMILRDEEVPALFTNKISPHQLGLADVLSALRFTGEFPKKLTLVGVIPESLEPHIGLTPTVEAMIEPALEQVLAALRESGVEAIPREAIHD
MRILVLGVGNILLTDEAIGVRIVEALEQRYILPDYVEILDGGTAGMELLGDMANRDHLIIADAIVSKKNAPGTMMILRDEEVPALFTNKISPHQLGLADVLSALRFTGEFPKKLTLVGVIPESLEPHIGLTPTVEAMIEPALEQVLAALRESGVEAIPRSDS--
MRILVLGVGNILLTDEAIGVRIVEALEQRYILPDYVEILDGGTAGMELLGDMANRDHLIIADAIVSKKNAPGTMMILRDEEVPALFTNKISPHQLGLADVLSALRFTGEFPKKLTLVGVIPESLEPHIGLTPTVEAMIEPALEQVLAALRESGVEAIPRSDS


Debugging use

In [57]:
dp = pd.DataFrame()
mask1, mask2, score = align(AF_sequence,jpred_sequence,debug=True,verbose=True)

print(score)
print(string(mask1))
print(string(mask2))

pd.DataFrame(dp,index=['-']+split(AF_sequence),columns=['-']+split(jpred_sequence))

max_row_val: 156, max_i: 162
max_col_val: 153, max_j: 161
Attempting semi-global alignment. \        Aim to align all of s1 against s2, \        or all of s2 against s1, whichever is longer.
Assigned 1 at index 161 of mask1 and index 161 of mask2.
13191
11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111100
111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111


,-,M,R,I,L,V,L,G,V,G,N,I,L,L,T,D,E,A,I,G,V,R,I,V,E,A,L,E,Q,R,Y,I,L,P,D,Y,V,E,I,L,D,G,G,T,A,G,M,E,L,L,G,D,M,A,N,R,D,H,L,I,I,A,D,A,I,V,S,K,K,N,A,P,G,T,M,M,I,L,R,D,E,E,V,P,A,L,F,T,N,K,I,S,P,H,Q,L,G,L,A,D,V,L,S,A,L,R,F,T,G,E,F,P,K,K,L,T,L,V,G,V,I,P,E,S,L,E,P,H,I,G,L,T,P,T,V,E,A,M,I,E,P,A,L,E,Q,V,L,A,A,L,R,E,S,G,V,E,A,I,P,R,S,D,S
-,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
M,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
R,0,-1,2,0,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-1,0,-2,-2,-2,-2,-1,0,-2,0,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-1,0,0,-2,0,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-1,0,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,0,-2,-2,-2,-2,-2,-2,-2,-2,0,-2,-2,-2
I,0,-1,0,3,1,-1,-3,-3,-3,-3,-3,-1,-3,-3,-3,-3,-3,-3,-1,-3,-3,-2,1,-1,-3,-3,-3,-3,-3,-2,-1,-1,-3,-3,-3,-3,-3,-3,-1,-3,-3,-3,-3,-3,-3,-3,-3,-2,-1,-3,-3,-3,-3,-2,-1,-2,-1,-3,-3,-1,-1,-3,-3,-3,-1,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-2,1,-1,-2,-1,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-1,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-2,-1,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-1,-3,-3,-3,-3,-3,-3,-3,-1,-3,-3,-3,-3,-3,-3,-3,-3,-3,0,-1,-3,-3,-3,-3,-3,-3,-3,-3,-3,-3,-2,-1,-3,-3,-3,-3,-3,-1,-3,-2,-1,-3,-3
L,0,-1,-2,1,4,2,0,-2,-4,-4,-4,-3,0,-2,-4,-4,-4,-4,-3,-2,-4,-4,-1,0,-2,-4,-2,-4,-4,-4,-3,-2,0,-2,-4,-4,-4,-4,-3,0,-2,-4,-4,-4,-4,-4,-4,-4,-1,0,-2,-4,-4,-4,-3,-2,-3,-2,-2,-3,-2,-2,-4,-4,-3,-2,-4,-4,-4,-4,-4,-4,-4,-4,-4,-4,-1,2,0,-2,-2,-4,-4,-4,-4,-2,-4,-4,-4,-4,-3,-2,-4,-4,-4,-2,-4,-2,-4,-4,-4,-2,-4,-4,-2,-4,-3,-2,-4,-4,-4,-4,-4,-4,-2,-4,-2,-4,-4,-4,-3,-2,-4,-4,-2,-4,-4,-4,-3,-2,-2,-4,-4,-4,-4,-4,-4,-4,-2,-1,-2,-4,-2,-4,-4,-4,-2,-4,-4,-2,-4,-3,-2,-4,-4,-4,-4,-3,-2,-4,-3,-2,-4
V,0,-1,-2,-1,2,5,3,1,-1,-3,-5,-5,-2,-1,-3,-5,-5,-5,-5,-4,-1,-3,-3,0,-1,-3,-4,-3,-5,-5,-5,-4,-2,-1,-3,-5,-3,-5,-5,-2,-1,-3,-5,-5,-5,-5,-5,-5,-3,-2,-1,-3,-5,-5,-5,-4,-3,-4,-3,-3,-4,-3,-3,-5,-5,-2,-3,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3,0,1,-1,-3,-3,-3,-5,-5,-4,-3,-5,-5,-5,-5,-4,-3,-5,-5,-4,-3,-4,-3,-5,-3,-4,-3,-5,-4,-3,-5,-4,-3,-5,-5,-5,-5,-5,-4,-3,-4,-1,-3,-3,-5,-4,-3,-5,-4,-3,-5,-5,-5,-4,-3,-3,-5,-5,-3,-5,-5,-5,-4,-3,-2,-3,-4,-3,-5,-3,-4,-3,-5,-4,-3,-5,-4,-3,-3,-5,-5,-5,-4,-3,-5,-4,-3
L,0,-1,-2,-3,0,3,6,4,2,0,-2,-4,-4,-1,-2,-4,-6,-6,-6,-6,-3,-2,-4,-2,-1,-2,-2,-4,-4,-6,-6,-6,-3,-3,-2,-4,-5,-4,-6,-4,-3,-2,-4,-6,-6,-6,-6,-6,-4,-2,-3,-2,-4,-6,-6,-6,-5,-4,-3,-4,-4,-5,-4,-4,-6,-4,-3,-4,-6,-6,-6,-6,-6,-6,-6,-6,-5,-2,-1,0,-2,-4,-4,-4,-6,-4,-5,-4,-6,-6,-6,-6,-5,-4,-6,-4,-5,-2,-4,-4,-5,-2,-4,-4,-4,-5,-4,-6,-5,-4,-6,-6,-6,-6,-4,-5,-2,-3,-2,-4,-4,-6,-5,-4,-4,-5,-4,-6,-6,-6,-3,-4,-4,-6,-5,-4,-6,-6,-6,-5,-4,-3,-2,-4,-4,-5,-2,-4,-4,-4,-5,-4,-6,-5,-4,-4,-6,-6,-6,-5,-4,-6,-5
G,0,-1,-2,-3,-2,1,4,7,5,3,1,-1,-3,-3,-2,-3,-5,-7,-7,-5,-5,-4,-3,-4,-3,-2,-3,-3,-5,-5,-7,-7,-5,-4,-4,-3,-5,-6,-5,-6,-5,-2,-1,-3,-5,-5,-7,-7,-6,-4,-1,-3,-3,-5,-7,-7,-7,-6,-5,-4,-5,-5,-6,-5,-5,-6,-5,-4,-5,-7,-7,-7,-5,-7,-7,-7,-7,-4,-3,-2,-1,-3,-5,-5,-5,-6,-5,-6,-5,-7,-7,-7,-7,-6,-5,-6,-3,-4,-3,-5,-5,-4,-3,-5,-5,-5,-6,-5,-5,-6,-5,-7,-7,-7,-6,-5,-4,-3,-2,-3,-5,-5,-7,-6,-5,-5,-6,-5,-7,-5,-5,-4,-5,-5,-7,-6,-5,-7,-7,-7,-6,-5,-4,-3,-5,-5,-4,-3,-5,-5,-5,-6,-5,-5